# 모듈

In [1]:
import pandas as pd
import numpy as np
import datetime

# 데이터

In [2]:
train_idf = pd.read_csv("data/train_identity.csv")
train_trans = pd.read_csv("data/train_transaction.csv")
test_idf = pd.read_csv("data/test_identity.csv")
test_trans = pd.read_csv("data/test_transaction.csv")

test_trans["isFraud"] = -1

train_trans = train_trans.merge(train_idf, on="TransactionID", how="left")
test_trans = test_trans.merge(train_idf, on="TransactionID", how="left")

train_trans["type"] = "train"
test_trans["type"] = "test"

df = pd.concat([train_trans, test_trans], axis=0)

start_date = datetime.datetime.strptime("2017.12.01", "%Y.%m.%d") # 시작일 설정
df["time"] = df["TransactionDT"].apply(
    lambda x: datetime.timedelta(seconds=x) + start_date
)
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month

del train_trans, test_trans

# Feature Engineering

## Count Encoding

In [3]:
cat_cols = ["ProductCD", "card1"]
for col in cat_cols:
    df[col + "_ce"] = df[col].map(df[col].value_counts(dropna=False))

## Aggregation

In [4]:
# ProductCD별로 TransactionAmt의 평균값과 표준편차를 구해 ex에 담음
ex = df.groupby("ProductCD").agg({"TransactionAmt" : ["mean", "std"]})
ex.reset_index(inplace=True)
ex.columns = ["ProductCD", "ProductCD_Amt_mean", "ProductCD_Amt_std"]

# 기존 데이터의 ProductCD를 기준으로 join
df = df.merge(ex, on="ProductCD", how="left")

## Account_make_D

In [5]:
# Account Start date를 만드는 함수 생성
def account_start_date(val):
    if np.isnan(val):
        return np.NaN
    else:
        days = int(str(val).split(".")[0])
        return pd.Timedelta(str(days) + " days")

# Account_make_Date 변수 생성
for i in ["D1", "D2", "D4", "D8", "D10", "D15"]:
    df["account_start_day"] = df[i].apply(account_start_date)
    df["account_make_date"] = (df["time"] - df["account_start_day"]).dt.date
    df["account_make_date_{}".format(i)] = (
        (10000 * pd.to_datetime(df["account_make_date"]).dt.year)
        + (100 * pd.to_datetime(df["account_make_date"]).dt.month)
        + (1 * pd.to_datetime(df["account_make_date"]).dt.day)
    )

ex = (
    df.groupby(["card1", "account_make_date_D1", "ProductCD"])
    .agg({"TransactionAmt" : ["mean", "std"]})
    .reset_index()
)

ex.columns = [
    "card1",
    "account_make_date_D1",
    "ProductCD",
    "D1_card_Product_mean",
    "D1_card_Product_std",
]

df = df.merge(ex, on=["card1", "account_make_date_D1", "ProductCD"], how="left")

## Prev/Next click, Amt feature

In [6]:
df["device_prev_trans"] = df["TransactionDT"] - df.groupby(["id_30", "id_31", "id_33", "DeviceType", "DeviceInfo"])["TransactionDT"].shift(1)
df["device_after_trans"] = df["TransactionDT"] - df.groupby(["id_30", "id_31", "id_33", "DeviceType", "DeviceInfo"])["TransactionDT"].shift(-1)

## LDA

- 카디널리티가 매우 높은 카테고리형 변수의 피쳐엔지니어링에 활용
- 각 카테고리를 토픽에 대한 확률 분포로 변환. 결과적으로 원본 카테고리를 K차원의 연속형 벡터로 변환
- 카테고리 간의 잠재적 관계를 포착

In [7]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
# LDA를 진행할 변수 선정
col1 = "card1"
col2 = "addr1"

temp = df[[col1, col2]]
col1col2_dict = {}

# col1을 키로, col2를 값 리스트로 하는 딕셔너리 생성
def col1col2(row):
    col1col2_dict.setdefault(row[col1], []).append(str(row[col2])) # 키가 없으면 빈 리스트 생성

# 각 행에 대해 col1col2 적용
temp.apply(lambda row: col1col2(row), axis=1)

# 각 card1에 대한 addr1들을 하나의 문자열로 변환
col1_keys = list(col1col2_dict.keys())
col1col2_dict_as_sentence = [" ".join(col1col2_dict[c]) for c in col1_keys]

# 각 문서(card1) 대해 각 단어(addr1)의 출현 빈도 계산
_as_matrix = CountVectorizer().fit_transform(col1col2_dict_as_sentence)

# card1을 5개의 토픽에 대한 확률 분포로 표현
# topics_of_col1 행렬의 각 행은 하나의 card1에 대한 5개 토픽의 확률 분포
topics_of_col1 = LDA(
                     n_components=5, # 토픽 수
                     n_jobs=-1, 
                     random_state=0 # 랜덤 시드
                     )
topics_of_col1 = topics_of_col1.fit_transform(_as_matrix)